In [10]:
import logging
import os
import pickle
import sys
import time
from pathlib import Path
from statistics import mean

import numpy as np
import polars as pl
import yaml
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


In [11]:
from pathlib import Path

import numpy as np
import tensorflow as tf
import yaml

from dvclive import Live
from dvclive.keras import DVCLiveCallback

params = Path("../params.yaml")
with open(params, "r") as file:
    params = yaml.safe_load(file)
DROPOUT = params["train"]["dropout"]
PATIENCE = params["train"]["patience"]
KERN_REG = params["train"]["kernel_regularizer"]
BATCH_NORMALIZATION = params["train"]["batch_normalization"]
BATCH_SIZE = params["train"]["batch_size"]
EPOCHS = params["train"]["epochs"]
LSTM_UNITS = params["train"]["lstm_units"]


def compile_and_fit(model, X_train, y_train, X_val, y_val, pitcher_name):
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=1e-4,  # - learning_rate: controls how much to change the model in response to the estimated error each time the model weights are updated.
        clipnorm=1.0,  # - clipnorm: clips gradients by norm; helps prevent exploding gradients.
        weight_decay=1e-4,  # - weight_decay: adds a penalty to the loss function to prevent overfitting.
    )
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_sparse_categorical_accuracy",
            patience=PATIENCE,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.2,
            patience=PATIENCE,
            min_lr=1e-6,
        ),
        DVCLiveCallback(live=Live(f"dvclive/{pitcher_name}_logs")),
    ]

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
    )

    return history


def create_model(input_shape, num_classes):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(shape=input_shape))
    model.add(
        tf.keras.layers.LSTM(
            LSTM_UNITS,
            return_sequences=True,
            dropout=DROPOUT,
            kernel_regularizer=tf.keras.regularizers.l2(KERN_REG),
        )
    )
    if BATCH_NORMALIZATION:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(
        tf.keras.layers.LSTM(
            LSTM_UNITS,
            return_sequences=True,
            dropout=DROPOUT,
            kernel_regularizer=tf.keras.regularizers.l2(KERN_REG),
        )
    )
    if BATCH_NORMALIZATION:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(
        tf.keras.layers.LSTM(
            LSTM_UNITS,
            dropout=DROPOUT,
            kernel_regularizer=tf.keras.regularizers.l2(KERN_REG),
        )
    )
    if BATCH_NORMALIZATION:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))
    return model


def calculate_class_weights(y):
    proportions = np.bincount(y) / len(y)
    for i, proportion in enumerate(proportions):
        if proportion == 0:
            proportions[i] = 1e-6
    inverseN = 1 / len(proportions)
    weights = [inverseN / proportion for proportion in proportions]
    return {i: w for i, w in enumerate(weights)}


In [12]:
df = pl.read_parquet(Path("../" + params["train"]["input_data_path"]))

In [13]:
logger = logging.getLogger("choo choo")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(
    logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
)
logger.addHandler(handler)

In [14]:
def create_sequences(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i : (i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [15]:
def create_training_data(pitcher_df, features):
    # sort the data by game_date, game_pk, at_bat_number, pitch_number
    pitcher_df = pitcher_df.sort(
        [
            "game_date",
            "game_pk",
            "at_bat_number",
            "pitch_number",
        ],
        descending=False,
    )

    # select features and target variable for initial split of X and y
    X = pitcher_df.select(pl.col(features)).to_numpy()
    y = pitcher_df.select(pl.col("next_pitch")).to_numpy().ravel()

    # encode target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y = y_encoded

    # declare time steps
    params_path = Path("../params.yaml")
    with open(params_path, "r") as file:
        params = yaml.safe_load(file)

    time_steps = params["train"]["time_steps"]

    X_seq, y_seq = create_sequences(X, y, time_steps)
    # convert to correct datatypes for model
    X_seq = X_seq.astype("float32")
    y_seq = y_seq.astype("int32")  # Ensure labels are integers

    # split into train, test, val
    train_split = params["train"]["train_split"]
    train_size = int(len(X_seq) * train_split)
    val_size = int(len(X_seq) * (1 - train_split) / 2)
    X_train, X_val, X_test = np.split(X_seq, [train_size, train_size + val_size])
    y_train, y_val, y_test = np.split(y_seq, [train_size, train_size + val_size])

    # scale features, no reason for minmax tbh, just read a stackoverflow that recommended it
    scaler = MinMaxScaler()
    n_samples, n_timesteps, n_features = X_train.shape
    X_train_reshaped = X_train.reshape(-1, n_features)
    X_train_scaled = scaler.fit_transform(X_train_reshaped)
    X_train_scaled = X_train_scaled.reshape(n_samples, n_timesteps, n_features)
    X_val_scaled = scaler.transform(X_val.reshape(-1, n_features)).reshape(X_val.shape)
    X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(
        X_test.shape
    )

    # logger.info("Unique labels in y_train:", list(np.unique(y_train)))
    # logger.info("Unique labels in y_val:", list(np.unique(y_val)))
    # logger.info("Unique labels in y_test:", list(np.unique(y_test)))

    return (
        X_train_scaled,
        y_train,
        X_val_scaled,
        y_val,
        X_test_scaled,
        y_test,
        label_encoder,
    )

In [16]:
def training_loop(df, params):
    pitcher_data = {}
    count = 0
    features = df.columns
    for feature in [
        "next_pitch",
        "pitcher",
        "player_name",
        "game_date",
        "game_pk",
        "pitch_type",
        "type",
    ]:
        features.remove(feature)
    start_time = time.time()
    for pitcher_df in df.group_by("pitcher"):
        pitcher_code = pitcher_df[0]
        pitcher_df = pitcher_df[1]

        pitcher_name = pitcher_df.select(pl.first("player_name")).item()
        num_pitches = len(pitcher_df)
        logger.info(
            f"Training model for pitcher: {pitcher_name} - {num_pitches} pitches"
        )
        X_train, y_train, X_val, y_val, X_test, y_test, label_encoder = (
            create_training_data(pitcher_df, features)
        )

        lstm_model = create_model(X_train.shape[1:], len(label_encoder.classes_))
        history = compile_and_fit(
            lstm_model,
            X_train,
            y_train,
            X_val,
            y_val,
            pitcher_name,
        )
        most_common_pitch_rate = pitcher_df.select(
            pl.col("pitch_type").value_counts().sort(descending=False).head(1)
        ).item()["count"] / len(pitcher_df)

        print(f"Most common pitch rate for {pitcher_name}: {most_common_pitch_rate}")
        test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
        pitcher_data[pitcher_code] = {
            "model": lstm_model,
            "history": history,
            "test_loss": test_loss,
            "test_accuracy": test_accuracy,
            "total_pitches": len(y_test) + len(y_train) + len(y_val),
            "unique_classes": len(np.unique(y_train)),
            "player_name": pitcher_name,
            "X_test": X_test,
            "y_test": y_test,
            "label_encoder": label_encoder,
            "most_common_pitch_rate": most_common_pitch_rate,
            "performance_gain": (test_accuracy - most_common_pitch_rate) * 100,
        }
        logger.info(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}")
        logger.info(
            f" Accuracy Gained over guessing most common pitch for {pitcher_name}: {pitcher_data[pitcher_code]["performance_gain"]:.2f}%",
        )
        logger.info(
            f"Average Test Accuracy: {mean([pitcher_data[pitcher]['test_accuracy'] for pitcher in pitcher_data])*100:.2f}%"
        )

        logger.info(
            f"Average Performance Gained over guessing most common pitch: {mean([pitcher_data[pitcher]['performance_gain'] for pitcher in pitcher_data]):.2f}%"
        )

        count += 1
        logger.info(
            f'{count} of {len(df.select(pl.col("pitcher")).unique())}, {(count/len(df.select(pl.col("pitcher")).unique())) * 100:.2f}% done!'
        )
    end_time = time.time()
    logger.info(f"Training took {end_time - start_time} seconds")
    return pitcher_data

In [17]:
pitcher_data = training_loop(df, params)

2024-12-31 20:04:02,603 - choo choo - INFO - Training model for pitcher: Scherzer, Max - 27260 pitches
2024-12-31 20:04:02,603 - choo choo - INFO - Training model for pitcher: Scherzer, Max - 27260 pitches


Epoch 1/200
298/298 ━━━━━━━━━━━━━━━━━━━━ 15:02 3s/step - loss: 4.1576 - sparse_categorical_accuracy: 0.093 ━━━━━━━━━━━━━━━━━━━━ 30s 103ms/step - loss: 4.1133 - sparse_categorical_accuracy: 0.10 ━━━━━━━━━━━━━━━━━━━━ 30s 102ms/step - loss: 4.0777 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 29s 101ms/step - loss: 4.0641 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 29s 101ms/step - loss: 4.0503 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 29s 102ms/step - loss: 4.0398 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 29s 102ms/step - loss: 4.0342 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 29s 101ms/step - loss: 4.0263 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 29s 101ms/step - loss: 4.0219 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 28s 100ms/step - loss: 4.0142 - sparse_categorical_accuracy: 0.11 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - loss: 4.0066 - sparse_categorical_accuracy: 0.1157 ━━━━━━━━━━━━━━━━━━━━ 2

Most common pitch rate for Scherzer, Max: 0.4962949376375642
128/128 ━━━━━━━━━━━━━━━━━━━━ 24s 195ms/step - loss: 2.2370 - sparse_categorical_accuracy: 0.50 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 2.2421 - sparse_categorical_accuracy: 0.4922 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 2.2560 - sparse_categorical_accuracy: 0.48 ━━━━━━━━━━━━━━━━━━━━ 8s 69ms/step - loss: 2.2590 - sparse_categorical_accuracy: 0.48 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - loss: 2.2599 - sparse_categorical_accuracy: 0.47 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - loss: 2.2603 - sparse_categorical_accuracy: 0.47 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - loss: 2.2584 - sparse_categorical_accuracy: 0.47 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - loss: 2.2558 - sparse_categorical_accuracy: 0.47 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 2.2503 - sparse_categorical_accuracy: 0.47 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - loss: 2.2461 - sparse_categorical_accuracy: 0.47 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - loss: 2.2412 - sparse_categorical_accurac

2024-12-31 20:18:12,646 - choo choo - INFO - Test Loss: 2.2806, Test Accuracy: 0.46
2024-12-31 20:18:12,646 - choo choo - INFO - Test Loss: 2.2806, Test Accuracy: 0.46
2024-12-31 20:18:12,647 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Scherzer, Max: -3.89%
2024-12-31 20:18:12,647 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Scherzer, Max: -3.89%
2024-12-31 20:18:12,648 - choo choo - INFO - Average Test Accuracy: 45.74%
2024-12-31 20:18:12,648 - choo choo - INFO - Average Test Accuracy: 45.74%
2024-12-31 20:18:12,648 - choo choo - INFO - Average Performance Gained over guessing most common pitch: -3.89%
2024-12-31 20:18:12,648 - choo choo - INFO - Average Performance Gained over guessing most common pitch: -3.89%
2024-12-31 20:18:12,651 - choo choo - INFO - 1 of 5, 20.00% done!
2024-12-31 20:18:12,651 - choo choo - INFO - 1 of 5, 20.00% done!
2024-12-31 20:18:12,658 - choo choo - INFO - Training model for pitcher: Cole, Gerrit 

Epoch 1/200
325/325 ━━━━━━━━━━━━━━━━━━━━ 16:41 3s/step - loss: 3.9156 - sparse_categorical_accuracy: 0.093 ━━━━━━━━━━━━━━━━━━━━ 29s 90ms/step - loss: 3.9216 - sparse_categorical_accuracy: 0.089 ━━━━━━━━━━━━━━━━━━━━ 30s 93ms/step - loss: 3.9150 - sparse_categorical_accuracy: 0.092 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - loss: 3.9127 - sparse_categorical_accuracy: 0.096 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - loss: 3.9060 - sparse_categorical_accuracy: 0.097 ━━━━━━━━━━━━━━━━━━━━ 29s 94ms/step - loss: 3.9052 - sparse_categorical_accuracy: 0.098 ━━━━━━━━━━━━━━━━━━━━ 30s 95ms/step - loss: 3.9062 - sparse_categorical_accuracy: 0.098 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - loss: 3.9050 - sparse_categorical_accuracy: 0.099 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - loss: 3.9011 - sparse_categorical_accuracy: 0.099 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - loss: 3.8981 - sparse_categorical_accuracy: 0.100 ━━━━━━━━━━━━━━━━━━━━ 30s 97ms/step - loss: 3.8954 - sparse_categorical_accuracy: 0.100 ━━━━━━━━━━━━━━━━━━━━ 30

...

Most common pitch rate for Gibson, Kyle: 0.17546879578083796
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - loss: 3.4703 - sparse_categorical_accuracy: 0.125 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - loss: 3.3595 - sparse_categorical_accuracy: 0.156 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - loss: 3.2889 - sparse_categorical_accuracy: 0.173 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - loss: 3.2798 - sparse_categorical_accuracy: 0.184 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - loss: 3.2611 - sparse_categorical_accuracy: 0.19 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - loss: 3.2423 - sparse_categorical_accuracy: 0.19 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - loss: 3.2185 - sparse_categorical_accuracy: 0.20 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - loss: 3.2028 - sparse_categorical_accuracy: 0.20 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - loss: 3.1894 - sparse_categorical_accuracy: 0.20 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - loss: 3.1796 - sparse_categorical_accuracy: 0.20 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - loss: 3.1714 - sparse_categorical_accu

2025-01-01 01:27:06,839 - choo choo - INFO - Test Loss: 3.1007, Test Accuracy: 0.26
2025-01-01 01:27:06,839 - choo choo - INFO - Test Loss: 3.1007, Test Accuracy: 0.26
2025-01-01 01:27:06,840 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Gibson, Kyle: 8.88%
2025-01-01 01:27:06,840 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Gibson, Kyle: 8.88%
2025-01-01 01:27:06,840 - choo choo - INFO - Average Test Accuracy: 43.30%
2025-01-01 01:27:06,840 - choo choo - INFO - Average Test Accuracy: 43.30%
2025-01-01 01:27:06,841 - choo choo - INFO - Average Performance Gained over guessing most common pitch: -0.09%
2025-01-01 01:27:06,841 - choo choo - INFO - Average Performance Gained over guessing most common pitch: -0.09%
2025-01-01 01:27:06,844 - choo choo - INFO - 4 of 5, 80.00% done!
2025-01-01 01:27:06,844 - choo choo - INFO - 4 of 5, 80.00% done!
2025-01-01 01:27:06,852 - choo choo - INFO - Training model for pitcher: Nola, Aaron - 270

Epoch 1/200
296/296 ━━━━━━━━━━━━━━━━━━━━ 15:58 3s/step - loss: 3.9569 - sparse_categorical_accuracy: 0.078 ━━━━━━━━━━━━━━━━━━━━ 25s 87ms/step - loss: 3.9665 - sparse_categorical_accuracy: 0.097 ━━━━━━━━━━━━━━━━━━━━ 24s 85ms/step - loss: 3.9860 - sparse_categorical_accuracy: 0.096 ━━━━━━━━━━━━━━━━━━━━ 24s 85ms/step - loss: 3.9727 - sparse_categorical_accuracy: 0.100 ━━━━━━━━━━━━━━━━━━━━ 24s 86ms/step - loss: 3.9634 - sparse_categorical_accuracy: 0.104 ━━━━━━━━━━━━━━━━━━━━ 24s 85ms/step - loss: 3.9598 - sparse_categorical_accuracy: 0.105 ━━━━━━━━━━━━━━━━━━━━ 24s 84ms/step - loss: 3.9549 - sparse_categorical_accuracy: 0.105 ━━━━━━━━━━━━━━━━━━━━ 24s 84ms/step - loss: 3.9537 - sparse_categorical_accuracy: 0.105 ━━━━━━━━━━━━━━━━━━━━ 24s 85ms/step - loss: 3.9567 - sparse_categorical_accuracy: 0.103 ━━━━━━━━━━━━━━━━━━━━ 24s 86ms/step - loss: 3.9567 - sparse_categorical_accuracy: 0.103 ━━━━━━━━━━━━━━━━━━━━ 24s 87ms/step - loss: 3.9570 - sparse_categorical_accuracy: 0.102 ━━━━━━━━━━━━━━━━━━━━ 24

Most common pitch rate for Nola, Aaron: 0.31185423841795357
127/127 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - loss: 2.1320 - sparse_categorical_accuracy: 0.312 ━━━━━━━━━━━━━━━━━━━━ 9s 75ms/step - loss: 2.0872 - sparse_categorical_accuracy: 0.304 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - loss: 2.0621 - sparse_categorical_accuracy: 0.30 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - loss: 2.0572 - sparse_categorical_accuracy: 0.30 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - loss: 2.0636 - sparse_categorical_accuracy: 0.28 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - loss: 2.0649 - sparse_categorical_accuracy: 0.28 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - loss: 2.0652 - sparse_categorical_accuracy: 0.27 ━━━━━━━━━━━━━━━━━━━━ 11s 99ms/step - loss: 2.0663 - sparse_categorical_accuracy: 0.2764 ━━━━━━━━━━━━━━━━━━━━ 11s 96ms/step - loss: 2.0649 - sparse_categorical_accuracy: 0.277 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - loss: 2.0648 - sparse_categorical_accuracy: 0.277 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - loss: 2.0639 - sparse_categorical

2025-01-01 02:12:23,171 - choo choo - INFO - Test Loss: 2.1318, Test Accuracy: 0.29
2025-01-01 02:12:23,171 - choo choo - INFO - Test Loss: 2.1318, Test Accuracy: 0.29
2025-01-01 02:12:23,171 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Nola, Aaron: -2.04%
2025-01-01 02:12:23,171 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Nola, Aaron: -2.04%
2025-01-01 02:12:23,172 - choo choo - INFO - Average Test Accuracy: 40.47%
2025-01-01 02:12:23,172 - choo choo - INFO - Average Test Accuracy: 40.47%
2025-01-01 02:12:23,173 - choo choo - INFO - Average Performance Gained over guessing most common pitch: -0.48%
2025-01-01 02:12:23,173 - choo choo - INFO - Average Performance Gained over guessing most common pitch: -0.48%
2025-01-01 02:12:23,179 - choo choo - INFO - 5 of 5, 100.00% done!
2025-01-01 02:12:23,179 - choo choo - INFO - 5 of 5, 100.00% done!
2025-01-01 02:12:23,181 - choo choo - INFO - Training took 22100.58839917183 seconds
202